# Creating a Python Dashboard

## By: ZXS107020

### Introduction:

Data science is the hot new topic in the information technology industry. We live in the age of data and with its exponential expansion comes the need for people who are equipped to interpret all of the information being collected. 

One important component of the data scientist's job that often gets overlooked is the background exploration and analysis that goes into understanding the data. This is the foundation of any data science experiment, and without it- the results an individual could come up with for a given set of data would most likely be useless. 

An important new tool that is becoming popular in the practition of this important step in any data science project is the rise of the dashboard. Dashboards are convenient ways to collect and clean your information and then present it in a neat, concise form using pictures and graphs. After all, as the saying goes, "a picture is worth 1,000 words."

### Getting Started

In order to create our dashbooard, we will be using the `dash` and `plotly` packages. Our input will be a .csv file containing OHLC data for various cryptocurrencies over time. For more instructions on how-to install the packages please see this [link](https://dash.plot.ly/installation). 

In [1]:
# Import the required libraries
import os
import pandas as pd
import dash
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_core_components as dcc
import plotly.graph_objs as go

Now that we have installed and imported all of our required libraries, lets go ahead and take a moment to go over our procedure. The pre-requisite for moving forward is having a basic understanding of the information contained in our data. This will allow us to structure our application appropriately. 

The modeling phase of the dashboard can be broken down further into three different components:  
1) The user interface (Model)  
2) The data visualizations (View)  
3) The interactions between the user inputs and graphical outputs (Controller)  

#### The Template:

This is the general structure of the `dash` dashboard that can be further customized for any use case.  

In [ ]:
###########################
# Data Manipulation / Model
###########################

def fetch_data(q):
    df = pd.read_sql(
        sql=q,
        con=conn
    )
    return df

#########################
# Dashboard Layout / View
#########################

# Set up Dashboard and create layout
app = dash.Dash()
app.css.append_css({
    "external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"
})

app.layout = html.Div([

    # Page Header
    html.Div([
        html.H1('Project Header')
    ]),

])

#############################################
# Interaction Between Components / Controller
#############################################

# Template
@app.callback(
    Output(component_id='selector-id', component_property='figure'),
    [
        Input(component_id='input-selector-id', component_property='value')
    ]
)
def ctrl_func(input_selection):
    return None


# start Flask server
if __name__ == '__main__':
    app.run_server(
        debug=True,
        host='0.0.0.0',
        port=8050
    )

#### The Model:

Once we have developed an understanding of our data, we must develop some sort of layout. We must chooses the variables and graphics that we wish to depict and decide how each of these components will fit together to form a larger product. This can be done mentally or even by hand using a pen and paper. Traditionally, the user interface would be designed as a web-application using CSS and HTML and other programming languages. However, the Python packages that we will be using will allow us to write all of the required code primarily in Python. 

Before we can do that, however, we will need to collect some information from the user to get started:  

In [ ]:
#######################################################################################################################
############################################### Collect user-inputs ###################################################
#######################################################################################################################

wd = input('Enter the path of the desired working directory: ')
fn = input('Enter the name of the input data file: ')

# Change the settings
os.chdir(wd)

# Load the data
data = pd.read_csv(fn)

In [ ]:
#######################################################################################################################
################################################## Pre-Processing #####################################################
#######################################################################################################################

# Fix the casing
data.columns = map(str.lower, data.columns)
data.columns = data.columns.str.replace(" ", "_")

# Convert the date
data['date'] = pd.to_datetime(data['date'])
data['date'].apply(lambda x: x.strftime('%m/%d/%Y'))

# Filter out currencies with less than two data points
data = data.groupby('currency').filter(lambda x: len(x) > 3)

In [ ]:
#######################################################################################################################
################################################### Interface #########################################################
#######################################################################################################################

# Function to select results of user input
def load_table(value):
    df = data[data['currency'] == value]
    
    return df

# Function to obtain results of user input
def gen_tab(df, max_rows = 10):
    return html.Table([html.Tr([html.Th(col) for col in df.columns])] +
                      [html.Tr([html.Td(df.iloc[i][col]) for col in df.columns]) for i in range(min(len(df), max_rows))])

# Initialize Dashboard
app = dash.Dash()

# Format the layout
app.layout = html.Div([
        # Page header
        html.Div([html.H1('Cryptocurrency Price Viewer:')]),
        
        # Dropdown menu
        dcc.Dropdown(id = 'dropdown',
                     options = [{'label': i, 'value': i} for i in data['currency'].unique()],
                     value = 'a'),
        
        # Results table
        html.Div(id = 'match-results'),

        # Price graph
        html.Div([dcc.Graph(id = 'graph')]),
        
        # User input
        html.Div(id = 'user-input', style = {'display': 'none'})])

# Callback to store user value    
@app.callback(
        Output(component_id = 'match-results', component_property = 'children'),
        [Input(component_id = 'dropdown', component_property = 'value')])

# Function to populate table based on user value
def disp_tab(value):
    dff = load_table(value)
    
    return gen_tab(dff)

In [ ]:
#######################################################################################################################
####################################################### Run ###########################################################
#######################################################################################################################

# Start the instance
if __name__ == '__main__':
    app.run_server(
        debug = False,
        host = '0.0.0.0',
        port = 8050)  